In [1]:
import torch
import os
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from MelodyLSTMEmbV2 import MelodyLSTMEmb

# Getting data

In [2]:
class ChordMelodyDataset(torch.utils.data.Dataset):
    def __init__(self, track_count, compass_count_path, X_path, y_path, window_size=10):
        self.window_size = window_size
        
        print('Loading compass count')
        self.compass_count = pd.read_csv(compass_count_path, names=['tid', 'compass_count'], nrows=track_count)
        self.cc = self.compass_count['compass_count'].to_numpy()
        self.cc_w = self.compass_count['compass_count'].to_numpy() - window_size
        self.cum_cc = np.cumsum(self.cc)
        self.cum_cc_w = np.cumsum(self.cc_w)
        
        self.cum_cc = np.append([0], self.cum_cc)
        self.cum_cc_w = np.append([0], self.cum_cc_w)

        
        print('Loading X')
        self.X = pd.read_csv(X_path, names=['n'+str(i) for i in range(36)] + ['chord', 'prev'], nrows=self.cum_cc[-1])
        
        print('Loading y')
        self.y = pd.read_csv(y_path, names=['n'+str(i) for i in range(36)], nrows=self.cum_cc[-1])
        
    def __len__(self):
        return self.cum_cc_w[-1]
    
    def __getitem__(self, idx):
        bucket = np.searchsorted(self.cum_cc_w, idx, side='right')
        # print(bucket)
        delta = idx - self.cum_cc_w[bucket - 1]
        # print(delta)
        idx_start = self.cum_cc[bucket - 1]
        # print(idx_start)
        idx = idx_start + delta
        
        # print(idx)
        x_data = torch.tensor(self.X.iloc[idx : idx + self.window_size].to_numpy(dtype=float))
        y_data = torch.tensor(self.y.iloc[idx : idx + self.window_size].to_numpy(dtype=float))

        return x_data.float(), y_data.float()

# Training

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device", device)

Device cuda


In [4]:
dataset = ChordMelodyDataset(100, '../data/compass_count2.csv', '../data/X2.csv', '../data/y2.csv', window_size=1)

Loading compass count
Loading X
Loading y


In [5]:
len(dataset)

79180

In [12]:
data_loader = torch.utils.data.DataLoader(dataset, batch_size=256)

In [13]:
num_epochs = 1
learning_rate = 1e-6

input_size = 56 #number of features
hidden_size = 512 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers

output_size = 36 #number of output classes 

In [14]:
mlstm = MelodyLSTMEmb(input_size, hidden_size, output_size, 
                   num_layers, device, threshold=0.6)
mlstm = mlstm.to(device)

In [15]:
criterion = torch.nn.BCELoss()
optimizer = torch.optim.AdamW(mlstm.parameters(), lr=learning_rate) 

In [16]:
for epoch in range(num_epochs):
    for X, y in tqdm(data_loader):
        X = X.cuda()
        y = y.cuda()
        outputs = mlstm.forward(X) #forward pass
        optimizer.zero_grad() #calculate the gradient, manually setting to 0
        
        # obtain the loss function
        # print(outputs.shape, y[:,-1,:].shape)
        loss = criterion(outputs, y[:,-1,:])

        loss.backward() #calculates the loss of the loss function

        optimizer.step() #improve from loss, i.e backprop
        
    # torch.save(mlstm.state_dict(), f'../models/muse2_t10000_bce_w32/checkpoint-{epoch:02}.pth')    
    print("Epoch: %d, loss: %1.8f" % (epoch, loss.item())) 

100%|████████████████████████████████████████████████████████████████████████████████| 310/310 [00:10<00:00, 30.45it/s]

Epoch: 0, loss: 0.68995893


In [17]:
arr = [np.random.randint(0, 2) for i in range(38)]

In [18]:
arr[-1] = 32
arr[-2] = 41

In [19]:
mlstm(torch.ones(1, 1, 38).to(device))

tensor([[0.5093, 0.4831, 0.5017, 0.4992, 0.4797, 0.4928, 0.4880, 0.5115, 0.4918,
         0.4877, 0.4880, 0.4864, 0.5137, 0.4956, 0.5151, 0.4896, 0.5145, 0.5146,
         0.4929, 0.5073, 0.4897, 0.5129, 0.4879, 0.5013, 0.5074, 0.4981, 0.4895,
         0.5133, 0.5073, 0.4927, 0.5127, 0.4935, 0.4887, 0.4990, 0.4839, 0.4995]],
       device='cuda:0', grad_fn=<SigmoidBackward0>)

In [20]:
mlstm(torch.zeros(1, 1, 38).to(device))

tensor([[0.5103, 0.4849, 0.5011, 0.4978, 0.4800, 0.4910, 0.4901, 0.5115, 0.4924,
         0.4878, 0.4857, 0.4864, 0.5121, 0.4940, 0.5137, 0.4883, 0.5134, 0.5154,
         0.4927, 0.5062, 0.4882, 0.5138, 0.4881, 0.4999, 0.5078, 0.4987, 0.4889,
         0.5118, 0.5076, 0.4907, 0.5123, 0.4917, 0.4883, 0.5004, 0.4851, 0.4988]],
       device='cuda:0', grad_fn=<SigmoidBackward0>)

In [21]:
mlstm(torch.reshape(torch.tensor(arr), (1, 1, -1)).to(device))

tensor([[0.5117, 0.4851, 0.5001, 0.4986, 0.4792, 0.4919, 0.4895, 0.5087, 0.4941,
         0.4889, 0.4875, 0.4880, 0.5125, 0.4962, 0.5123, 0.4897, 0.5154, 0.5154,
         0.4930, 0.5073, 0.4899, 0.5121, 0.4881, 0.5003, 0.5081, 0.4990, 0.4897,
         0.5121, 0.5093, 0.4923, 0.5116, 0.4909, 0.4880, 0.4985, 0.4861, 0.4998]],
       device='cuda:0', grad_fn=<SigmoidBackward0>)

In [50]:
t = torch.reshape(torch.tensor(arr), (1, 1, -1))

In [51]:
a = t[:, :, -2:]

In [52]:
t = t[:, :, :-2]

In [ ]:
torch.save(mlstm.state_dict(), '../models/muse2_t2000_v1.pth')

In [ ]:
mlstm.embedding(torch.tensor([1]).cuda())

In [ ]:
with open('../data/chords/CHORD_DICT.pickle', 'rb') as f: 
    CHORD_DICT = pickle.load(f)

In [ ]:
with open('../data/chords_reduced/CHORD_TO_EMB.pickle', 'rb') as f:
    CHORD_TO_EMB = pickle.load(f)

In [ ]:
CHORD_TO_EMB

In [ ]:
embeddings = mlstm.embedding.weight.data.cpu().numpy()

In [ ]:
embeddings[51]